In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install evaluate torch transformers bert-score rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00
   ━━━

## **0. 모델 불러오기**

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
import pandas as pd
from evaluate import load
import math
from tqdm import tqdm
import random


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# 모델과 토크나이저 경로
model_path = "/content/drive/MyDrive/7th-project/model/koT5/best_model2"

# 모델 로드
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 모델을 GPU로 이동
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("모델과 토크나이저가 성공적으로 로드되었습니다!")
print(f"모델 임베딩 크기: {model.config.vocab_size}")
print(f"토크나이저 단어 사전 크기: {len(tokenizer)}")

모델과 토크나이저가 성공적으로 로드되었습니다!
모델 임베딩 크기: 32112
토크나이저 단어 사전 크기: 32112


In [5]:
### 확인
print("토크나이저 특수 토큰:", tokenizer.special_tokens_map)
print("추가된 특수 토큰:", tokenizer.additional_special_tokens)
print("토크나이저 크기:", len(tokenizer))

토크나이저 특수 토큰: {'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_2>', '<extra_id_64>', '<extra_id_8>', '<extra_id_15>', '<character>', '<extra_id_7>', '<extra_id_67>', '<extra_id_38>', '<extra_id_30>', '<extra_id_69>', '<extra_id_14>', '<prediction>', '<extra_id_16>', '<extra_id_35>', '<extra_id_40>', '<extra_id_82>', '<extra_id_59>', '<extra_id_26>', '<extra_id_79>', '<extra_id_0>', '<extra_id_36>', '<extra_id_41>', '<extra_id_9>', '<extra_id_72>', '<extra_id_80>', '<extra_id_57>', '<extra_id_74>', '<setting>', '<extra_id_93>', '<extra_id_23>', '<extra_id_96>', '<action>', '<extra_id_4>', '<extra_id_11>', '<extra_id_77>', '<extra_id_87>', '<extra_id_43>', '<extra_id_88>', '<extra_id_60>', '<extra_id_99>', '<extra_id_71>', '<feeling>', '<outcomeResolution>', '<extra_id_12>', '<extra_id_37>', '<extra_id_62>', '<causalRelationship>', '<extra_id_18>', '<extra_id_63>', '<extra_id_32>', '<name>', '<extra_id_53>', '<extra_id_68>', '<extr

# **1. 데이터 준비**
- 학습할 때 사용했던 데이터 형식과 동일하도록

## **1-1. 데이터 불러오기**

In [6]:
val_data = pd.read_csv("/content/drive/MyDrive/7th-project/data/dataset_val.csv")

In [7]:
val_data.head()

,id,img_path,caption,srcText,name,i_action,classification,character,setting,action,feeling,causalRelationship,outcomeResolution,prediction
0,03_02T_03S_9788998212643_54879,/content/data/VL_02T_자연탐구_03S_초등_고하...,횡단보도로 길을 건너는 사람들의 집단,“출발합니다! 손잡이 꼭 잡으세요.” 삐이익. 다음 정류장에 내릴 손님이 벨을 눌렀...,신호등,초록색이다,자연탐구,"손잡이,정류장,손님,벨,차돌,신호등",NaN,"출발합니다,잡으세요,내릴,눌렀어요,멈췄다 가자,지키는,지켜야 해요",NaN,NaN,NaN,NaN
1,03_02T_03S_9788967760144_35692,/content/data/VL_02T_자연탐구_03S_초등_고하...,보트 위에 있는 남자와 여자의 그림,두 번째로 간 곳은 바다였다. “이번 모험은 고기잡이배에서 생활하는 것이다.” 마음...,고기잡이배,바다에 떠 있다,자연탐구,"모험,나는,그물,물고기,파도","바다,고기잡이배,하루 종일","출발했다,생활하는,간,태운,먹지 못하고,끌어 올려야",NaN,NaN,NaN,멀고 먼 바다로 출발했다.
2,03_02T_03S_9788998212636_5162,/content/data/VL_02T_자연탐구_03S_초등_고하...,들판에 있는 동물 무리의 그림,제주도에 여행 온 친구들은 힝힝 씨가 들려주는 제주도 이야기를 아주 좋아해요. “제...,해녀,바다에 있다,자연탐구,"친구,힝힝 씨,해녀,섬",제주도,들려주는,좋아해요,NaN,NaN,NaN
3,03_02T_03S_9791128211331_32163,/content/data/VL_02T_자연탐구_03S_초등_고하...,침대에서 자는 아이의 그림,쿠르쿠르는 에코스 덕분에 아주 오랜만에 단잠을 잘 수 있었어. 하지만 단 하루뿐이었...,난쟁이,침대에 누워있다,자연탐구,"쿠르쿠르,에코스",다음 날,"잘 수 있었어,들리기 시작한 거야",끔찍한,NaN,NaN,NaN
4,03_02T_03S_9791128211195_59242,/content/data/VL_02T_자연탐구_03S_초등_고하...,만화 캐릭터가 있는 빨간색과 주황색 배경,“이 원반은 사람의 핏속에 있는 적혈구랍니다. 적혈구는 사람의 몸속 구석구석으로 산...,적혈구,붉은색 원반 모양을 하고 흘러가고 있어요.,자연탐구,"원반,사람,적혈구,산소",핏속,"나르지요,여행할 거예요",NaN,NaN,NaN,NaN


## **1-2. text 전처리**





### **input text**
- special token을 활용하여 각 정보를 구분
  - 필수 필드
    - 결측치 확인(결측치 허용 x)
  - 선택 필드
    - 다중 아이템의 경우 (,)로 구분 후 각 항목에 토큰 부여
    - 결측치가 있는 경우 `<empty>` 토큰 부여
- caption과 보조적 정보를 포함하여 구성
  - 행별 고유 시드 설정: 각 행의 id(또는 index)를 seed로 설정   
  → 동일한 데이터를 사용하면 언제나 같은 결과가 나오도록 보장
  - 랜덤 순서 보장: 각 행마다 token의 순서를 랜덤하게 설정  
  → 모델이 token 간 순서나 위치에 지나치게 의존하지 않고, 각 요소의 의미와 역할을 학습하도록 유도

In [8]:
import random
import hashlib

In [9]:
### Input Text 생성 함수

def generate_input_text(row):
    ## 고유 seed 설정(해시 기반)
    seed_value = int(hashlib.md5(str(row["id"]).encode()).hexdigest(), 16) % (10 ** 8)
    random.seed(seed_value)


    ## 필수 필드
    # 존재성 보장(assertion)
    required_fields = ["caption", "name", "i_action", "classification"]
    for field in required_fields:
        assert pd.notna(row[field]) and row[field].strip(), f"Error: '{field}' 필드는 비워둘 수 없습니다."

    required_fields = [
        f"<caption> {row['caption'].strip()}",
        f"<name> {row['name'].strip()}",
        f"<i_action> {row['i_action'].strip()}",
        f"<classification> {row['classification'].strip()}"
    ]


    ## 선택 필드
    optional_fields = {
        "character": "<character>",
        "setting": "<setting>",
        "action": "<action>",
        "feeling": "<feeling>",
        "causalRelationship": "<causalRelationship>",
        "outcomeResolution": "<outcomeResolution>",
        "prediction": "<prediction>"
    }

    optional_tokens = []
    for field, token in optional_fields.items():
        if pd.notna(row[field]) and row[field].strip():
            # 여러 항목 처리(쉼표로 구분)
            items = [item.strip() for item in row[field].split(",")]
            optional_tokens.extend([f"{token} {item}" for item in items])
        else:
            optional_tokens.append(f"{token} <empty>")


    ## 토큰 순서 섞기
    all_tokens = required_fields + optional_tokens
    random.shuffle(all_tokens)


    return "generate: " + " ".join(all_tokens) # prefix 추가

In [10]:
## Input Text 생성
val_data["input_text"] = val_data.apply(generate_input_text, axis = 1)

### **output text**

In [11]:
import re

In [12]:
### Output Text 전처리 함수

def preprocess_output_text(story):
    if pd.isna(story):
        return "<empty>"

    story = re.sub(r'[^\w\s가-힣.,?!]', '', story) # 특수 문자 제거(문장 부호 유지)

    story = re.sub(r'\s+', ' ', story).strip() # 불필요한 공백 제거

    return story

In [13]:
## Output Text 전처리
val_data["output_text"] = val_data["srcText"].apply(preprocess_output_text)

### **데이터 확인**  

In [14]:
val_data[["id", "input_text", "output_text"]].head()

,id,input_text,output_text
0,03_02T_03S_9788998212643_54879,generate: <character> 손님 <action> 눌렀어요 <charac...,출발합니다! 손잡이 꼭 잡으세요. 삐이익. 다음 정류장에 내릴 손님이 벨을 눌렀어요...
1,03_02T_03S_9788967760144_35692,generate: <character> 나는 <caption> 보트 위에 있는 남자...,두 번째로 간 곳은 바다였다. 이번 모험은 고기잡이배에서 생활하는 것이다. 마음의 ...
2,03_02T_03S_9788998212636_5162,generate: <classification> 자연탐구 <character> 힝힝...,제주도에 여행 온 친구들은 힝힝 씨가 들려주는 제주도 이야기를 아주 좋아해요. 제주...
3,03_02T_03S_9791128211331_32163,generate: <i_action> 침대에 누워있다 <prediction> <em...,쿠르쿠르는 에코스 덕분에 아주 오랜만에 단잠을 잘 수 있었어. 하지만 단 하루뿐이었...
4,03_02T_03S_9791128211195_59242,generate: <causalRelationship> <empty> <outcom...,이 원반은 사람의 핏속에 있는 적혈구랍니다. 적혈구는 사람의 몸속 구석구석으로 산소...


# **2.Dataset 및 DataLoader 구성**
- model을 학습시키기 위한 데이터셋 구축 및 토큰화

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json

In [16]:
## 특수 token 정의
special_tokens = [
    "<caption>", "<name>", "<i_action>", "<classification>", "<character>",
    "<setting>", "<action>", "<feeling>", "<causalRelationship>",
    "<outcomeResolution>", "<prediction>", "<empty>"
]

In [17]:
print("Special tokens added:", tokenizer.special_tokens_map)
print("Vocabulary size:", len(tokenizer))

special_token_ids = tokenizer.convert_tokens_to_ids(special_tokens)
print("Special Token IDs:", sorted(special_token_ids))

Special tokens added: {'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_2>', '<extra_id_64>', '<extra_id_8>', '<extra_id_15>', '<character>', '<extra_id_7>', '<extra_id_67>', '<extra_id_38>', '<extra_id_30>', '<extra_id_69>', '<extra_id_14>', '<prediction>', '<extra_id_16>', '<extra_id_35>', '<extra_id_40>', '<extra_id_82>', '<extra_id_59>', '<extra_id_26>', '<extra_id_79>', '<extra_id_0>', '<extra_id_36>', '<extra_id_41>', '<extra_id_9>', '<extra_id_72>', '<extra_id_80>', '<extra_id_57>', '<extra_id_74>', '<setting>', '<extra_id_93>', '<extra_id_23>', '<extra_id_96>', '<action>', '<extra_id_4>', '<extra_id_11>', '<extra_id_77>', '<extra_id_87>', '<extra_id_43>', '<extra_id_88>', '<extra_id_60>', '<extra_id_99>', '<extra_id_71>', '<feeling>', '<outcomeResolution>', '<extra_id_12>', '<extra_id_37>', '<extra_id_62>', '<causalRelationship>', '<extra_id_18>', '<extra_id_63>', '<extra_id_32>', '<name>', '<extra_id_53>', '<extra_id_68>

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorForSeq2Seq

In [19]:
### masking 함수 정의

def special_token_masking(input_ids, attention_mask, special_token_ids):
    mask_indices = (input_ids.unsqueeze(-1) == torch.tensor(special_token_ids)).any(dim=-1)
    attention_mask[mask_indices] = 0

    return attention_mask

In [20]:
### Dataset 클래스 정의

class StoryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, special_token_ids, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.special_token_ids = special_token_ids

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        input_encoding = self.tokenizer(
            row["input_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        target_encoding = self.tokenizer(
            row["output_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids = input_encoding["input_ids"].squeeze(0)
        attention_mask = input_encoding["attention_mask"].squeeze(0)
         # Special Token 마스킹 적용
        attention_mask = special_token_masking(input_ids, attention_mask, self.special_token_ids)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": target_encoding["input_ids"].squeeze(0),
        }

In [21]:
from transformers import DataCollatorForSeq2Seq

val_dataset = StoryDataset(val_data, tokenizer, special_token_ids)

data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)
val_dataloader = DataLoader(val_dataset, batch_size = 8, collate_fn = data_collator)

In [22]:
### 확인
for batch in val_dataloader:
    print("Batch keys:", batch.keys())
    print("input_ids shape:", batch["input_ids"].shape)  # (batch_size, max_length)
    print("labels shape:", batch["labels"].shape)  # (batch_size, max_length)
    break

Batch keys: dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])
input_ids shape: torch.Size([8, 128])
labels shape: torch.Size([8, 128])


/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [23]:
for token in special_tokens:
    token_id = tokenizer.convert_tokens_to_ids(token)
    print(f"Token: {token}, Token ID: {token_id}, Decoded: {tokenizer.decode([token_id])}")

Token: <caption>, Token ID: 32108, Decoded: <caption>
Token: <name>, Token ID: 32107, Decoded: <name>
Token: <i_action>, Token ID: 32109, Decoded: <i_action>
Token: <classification>, Token ID: 32110, Decoded: <classification>
Token: <character>, Token ID: 32100, Decoded: <character>
Token: <setting>, Token ID: 32102, Decoded: <setting>
Token: <action>, Token ID: 32103, Decoded: <action>
Token: <feeling>, Token ID: 32104, Decoded: <feeling>
Token: <causalRelationship>, Token ID: 32106, Decoded: <causalRelationship>
Token: <outcomeResolution>, Token ID: 32105, Decoded: <outcomeResolution>
Token: <prediction>, Token ID: 32101, Decoded: <prediction>
Token: <empty>, Token ID: 32111, Decoded: <empty>


In [24]:
# 샘플 데이터 토큰화 확인
sample_text = val_data["input_text"].iloc[0]
input_ids = tokenizer(sample_text, return_tensors="pt").input_ids
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=False)

print(f"Original Input: {sample_text}")
print(f"Tokenized IDs: {input_ids.tolist()}")
print(f"Decoded Tokens: {decoded_text}")


Original Input: generate: <character> 손님 <action> 눌렀어요 <character> 차돌 <i_action> 초록색이다 <action> 출발합니다 <action> 지키는 <name> 신호등 <caption> 횡단보도로 길을 건너는 사람들의 집단 <feeling> <empty> <causalRelationship> <empty> <prediction> <empty> <character> 손잡이 <character> 벨 <classification> 자연탐구 <action> 내릴 <action> 지켜야 해요 <setting> <empty> <outcomeResolution> <empty> <character> 정류장 <character> 신호등 <action> 멈췄다 가자 <action> 잡으세요
Tokenized IDs: [[11447, 1537, 845, 13407, 26202, 25889, 32100, 4122, 25889, 32103, 5603, 26628, 295, 25889, 32100, 130, 26192, 25889, 32109, 10326, 26296, 41, 25889, 32103, 2212, 560, 25889, 32103, 8922, 25889, 32107, 7497, 25983, 25889, 32108, 15488, 25933, 647, 2527, 3590, 25893, 3433, 3967, 25889, 32104, 25889, 32111, 25889, 32106, 25889, 32111, 25889, 32101, 25889, 32111, 25889, 32100, 250, 9793, 25889, 32100, 3321, 25889, 32110, 1050, 26656, 25948, 25889, 32103, 6865, 25889, 32103, 15455, 4455, 25889, 32102, 25889, 32111, 25889, 32105, 25889, 32111, 25889, 32100, 16387, 25935

# **3. 평가**

In [25]:
import torch
from tqdm import tqdm

# 모델 예측 함수
def generate_predictions(model, tokenizer, dataloader):
    model.eval()
    preds, references = [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Generating Predictions"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # 모델이 생성한 문장 예측(최적 조합 설정)
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128,
                num_beams=3,
                length_penalty=0.8,
                repetition_penalty=1.5,
                no_repeat_ngram_size=3,
                early_stopping=True
            )

            batch_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            batch_actuals = tokenizer.batch_decode(labels, skip_special_tokens=True)

            preds.extend(batch_preds)
            references.extend(batch_actuals)

    return preds, references

## **3-1. BERTScore**
- 단어 임베딩을 활용하여 의미적으로 유사한 단어를 평가
  - 문장의 의미적 유사도를 고려하기 위해 BERTScore 사용

- BLEU나 ROUGE는 n-gram 기반 지표, 단어의 위치나 의미를 반영 X

In [26]:
from evaluate import load
import numpy as np

# BERTScore 로드
bertscore = load("bertscore")

def compute_bertscore(preds, references):
    bertscore_result = bertscore.compute(predictions=preds, references=references, lang="ko")
    bertscore_f1 = np.mean(bertscore_result["f1"])  # F1-score 사용
    print(f"\nBERTScore F1: {bertscore_f1:.4f}**")
    return bertscore_f1


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [29]:
# 실행
preds, references = generate_predictions(model, tokenizer, val_dataloader)
bertscore_f1 = compute_bertscore(preds, references)

# BERTScore가 단일 값이면 그대로 출력
if isinstance(bertscore_f1, (float, np.float64)):
    print(f"BERTScore F1: {bertscore_f1:.4f}")
# 리스트나 배열이면 평균 계산
else:
    print(f"BERTScore F1: {np.mean(bertscore_f1):.4f}")

Generating Predictions: 100%|██████████| 625/625 [22:27<00:00,  2.16s/it]



BERTScore F1: 0.8071**
BERTScore F1: 0.8071


## **3-2. METEOR**


In [30]:
from evaluate import load

# METEOR Metric 불러오기
meteor = load("meteor")

# METEOR 점수 계산 함수
def compute_meteor(preds, references):
    scores = meteor.compute(predictions=preds, references=references)
    return scores["meteor"]


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [31]:
# METEOR 점수 계산
meteor_score = compute_meteor(preds, references)

# 결과 출력
print(f"METEOR Score: {meteor_score:.4f}")

METEOR Score: 0.3458


## **3-3. CIDEr**
- n-gram 가중치를 기반으로 문장의 일관성과 유사성을 평가

In [32]:
!pip install "git+https://github.com/salaniz/pycocoevalcap.git"

  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-30lde5w5
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-30lde5w5
  Resolved https://github.com/salaniz/pycocoevalcap.git to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=c6255a952aacbb19824ee31b21f3a32a85bff520ed3500a8079ba4ac1413aef7
  Stored in directory: /tmp/pip-ephem-wheel-cache-sijhqled/wheels/e5/d1/50/82763a91172a5c8058c9efff8692f3a41570e3ddd5b5b2c4b4
Successfully built pycocoevalcap


In [33]:
import sys
sys.path.append("coco-caption")

from pycocoevalcap.cider.cider import Cider


# CIDEr 점수 계산 함수
def compute_cider(preds, references):
    scorer = Cider()
    cider_score, _ = scorer.compute_score({i: [p] for i, p in enumerate(preds)}, {i: [r] for i, r in enumerate(references)})
    return cider_score




In [34]:
# CIDEr 평가 실행
cider_score = compute_cider(preds, references)

# 결과 출력
print(f"CIDEr Score: {cider_score:.4f}")

CIDEr Score: 0.9758


## **3-4. SPICE**
- 구문 트리(Dependency Tree)를 활용하여 의미적 유사성을 평가

In [35]:
from pycocoevalcap.spice.spice import Spice


# SPICE 점수 계산 함수
def compute_spice(preds, references):
    scorer = Spice()
    spice_score, _ = scorer.compute_score({i: [p] for i, p in enumerate(preds)}, {i: [r] for i, r in enumerate(references)})
    return spice_score


In [36]:
# SPICE 평가 실행
spice_score = compute_spice(preds, references)

# 결과 출력
print(f"SPICE Score: {spice_score:.4f}")

Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.
SPICE Score: 0.1921


## **3-5. 결과 해석**
1. ```BERTScore F1: 0.8071```: 생성된 문장이 정답과 높은 의미적 유사성을 보임

2. ```METEOR Score: 0.3458```: 형태소/어순이 꽤 유사하지만, 개선 여지가 있음
3. ```CIDEr Score: 0.9758```: 정답 문장에서 중요한 단어들을 잘 포함하고 있음

4. ```SPICE Score: 0.192```: 의미 구조가 완전히 매칭되지 않음 (논리적 흐름 개선 필요)

## **3-6. 디코딩 파라미터 조정**
- Beam Search 개수, 길이 패널티, 반복 패널티 등 조정 실험

In [ ]:
from itertools import product

# 하이퍼파라미터 조합 설정
num_beams_list = [3, 5, 7]  # Beam Search 개수
length_penalty_list = [0.8, 1.0, 1.2]  # 길이 패널티
repetition_penalty_list = [1.2, 1.5, 2.0]  # 반복 패널티
no_repeat_ngram_size_list = [2, 3]  # 반복 방지 n-gram 크기

# 모든 조합 생성
param_combinations = list(product(num_beams_list, length_penalty_list, repetition_penalty_list, no_repeat_ngram_size_list))

print(f"총 {len(param_combinations)}개의 조합을 테스트합니다.")

총 54개의 조합을 테스트합니다.


In [ ]:
import random
from torch.utils.data import Subset, DataLoader

# 샘플링할 데이터 개수
sample_size = 100

# 샘플링할 인덱스 랜덤 선택
sample_indices = random.sample(range(len(val_dataset)), sample_size)

# 샘플 데이터셋 생성
sample_dataset = Subset(val_dataset, sample_indices)

# DataLoader 생성
sample_dataloader = DataLoader(sample_dataset, batch_size=8, shuffle=False, collate_fn=data_collator)

# 확인
print(f"샘플 데이터 개수: {len(sample_dataset)}")

샘플 데이터 개수: 100


In [ ]:
# 하이퍼파라미터 조합별 문장 생성 테스트 함수
def test_decoding_variants(model, tokenizer, dataloader, param_combinations):
    model.eval()
    results = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            for params in param_combinations:
                num_beams, length_penalty, repetition_penalty, no_repeat_ngram_size = params

                generated_ids = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_length=128,
                    num_beams=num_beams,
                    length_penalty=length_penalty,
                    repetition_penalty=repetition_penalty,
                    no_repeat_ngram_size=no_repeat_ngram_size,
                    early_stopping=True,
                )

                generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

                results.append({
                    "num_beams": num_beams,
                    "length_penalty": length_penalty,
                    "repetition_penalty": repetition_penalty,
                    "no_repeat_ngram_size": no_repeat_ngram_size,
                    "generated_texts": generated_texts
                })

    return results


In [ ]:
# 평가 함수
from evaluate import load

def evaluate_generation(preds, references):
    # BERTScore 계산
    bert_scores = bertscore.compute(predictions=preds, references=references, lang="ko")["f1"]
    avg_bert_score = sum(bert_scores) / len(bert_scores)

    # METEOR 계산
    meteor_score = meteor.compute(predictions=preds, references=references)["meteor"]

    # CIDEr 계산
    cider_score = compute_cider(preds, references)

    # SPICE 계산
    spice_score = compute_spice(preds, references)

    return avg_bert_score, meteor_score, cider_score, spice_score


In [ ]:
# 테스트 실행
decoding_results = test_decoding_variants(model, tokenizer, sample_dataloader, param_combinations)

In [ ]:
# 평가 결과
evaluation_results = []

for res in decoding_results:
    generated_texts = res["generated_texts"]

    # 정답 가져오기
    references = val_data["output_text"].iloc[sample_indices[:len(generated_texts)]].tolist()

    # 평가 점수 계산
    bert_score, meteor_score, cider_score, spice_score = evaluate_generation(generated_texts, references)

    # 결과 저장
    evaluation_results.append({
        "num_beams": res["num_beams"],
        "length_penalty": res["length_penalty"],
        "repetition_penalty": res["repetition_penalty"],
        "no_repeat_ngram_size": res["no_repeat_ngram_size"],
        "BERTScore": bert_score,
        "METEOR": meteor_score,
        "CIDEr": cider_score,
        "SPICE": spice_score,
        "Generated Texts": generated_texts
    })

# DataFrame으로 변환
df_results = pd.DataFrame(evaluation_results)

# 결과 확인
print("\n=== 하이퍼파라미터 튜닝 결과 (상위 5개, BERTScore 기준) ===")
print(df_results.sort_values(by="BERTScore", ascending=False).head(5).to_string(index=False))

print("\n=== 하이퍼파라미터 튜닝 결과 (상위 5개, METEOR 기준) ===")
print(df_results.sort_values(by="METEOR", ascending=False).head(5).to_string(index=False))

print("\n=== 하이퍼파라미터 튜닝 결과 (상위 5개, CIDEr 기준) ===")
print(df_results.sort_values(by="CIDEr", ascending=False).head(5).to_string(index=False))

print("\n=== 하이퍼파라미터 튜닝 결과 (상위 5개, SPICE 기준) ===")
print(df_results.sort_values(by="SPICE", ascending=False).head(5).to_string(index=False))

Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.

=== 하이퍼파라미터 튜닝 결과 (상위 5개, BERTScore 기준) ===
 num_beams  length_penalty  repetition_penalty  no_repeat_ngram_size  BERTScore   METEOR    CIDEr    SPICE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Generated Texts
         3             1.2                 1.5                     2   0.840346 0.430577 1.812495 0.284371                                          

In [ ]:
import pandas as pd

# 파일 경로 설정
file_path = "/content/drive/MyDrive/7th-project/jeonga/decoding_tuning_results.csv"

# 데이터프레임 CSV 저장
df_results.to_csv(file_path, index=False, encoding="utf-8-sig")

print(f"하이퍼파라미터 튜닝 결과 저장: {file_path}")


하이퍼파라미터 튜닝 결과 저장: /content/drive/MyDrive/7th-project/jeonga/decoding_tuning_results.csv


In [ ]:
# 방법1. 가중치 설정

import pandas as pd
from scipy.stats import zscore


# Z-score 정규화 (평균 0, 표준편차 1로 변환)
df_results["BERTScore_z"] = zscore(df_results["BERTScore"])
df_results["METEOR_z"] = zscore(df_results["METEOR"])
df_results["CIDEr_z"] = zscore(df_results["CIDEr"])
df_results["SPICE_z"] = zscore(df_results["SPICE"])

# 가중치 설정
weights = {
    "BERTScore": 0.4,  # 의미적 유사성
    "METEOR": 0.2,  # 형태소 및 어순
    "CIDEr": 0.2,  # 핵심 단어 반영(이미지 캡셔닝 위주 지표)
    "SPICE": 0.2,  # 문장 구조 평가
}

# 가중 평균 점수 계산
df_results["Final_Score"] = (
    df_results["BERTScore_z"] * weights["BERTScore"] +
    df_results["METEOR_z"] * weights["METEOR"] +
    df_results["CIDEr_z"] * weights["CIDEr"] +
    df_results["SPICE_z"] * weights["SPICE"]
)

# 최적의 하이퍼파라미터 조합 출력
best_params = df_results.sort_values(by="Final_Score", ascending=False).head(1)

print("\n=== 최적의 하이퍼파라미터 조합 ===")
print(best_params[["num_beams", "length_penalty", "repetition_penalty", "no_repeat_ngram_size", "BERTScore", "METEOR", "CIDEr", "SPICE", "Final_Score"]].to_string(index=False))



=== 최적의 하이퍼파라미터 조합 ===
 num_beams  length_penalty  repetition_penalty  no_repeat_ngram_size  BERTScore   METEOR    CIDEr    SPICE  Final_Score
         7             1.0                 1.5                     3   0.835798 0.429711 1.762743 0.318842     3.545836


In [ ]:
# 방법2. 각 지표에서 TOP 5 안에 드는 조합 중 평균 점수 가장 높은 조합

top_n = 5  # 각 지표별 상위 몇 개 볼지 결정

# 각 지표에서 상위 top_n 개 조합을 선택
top_bertscore = df_results.nlargest(top_n, "BERTScore")
top_meteor = df_results.nlargest(top_n, "METEOR")
top_cider = df_results.nlargest(top_n, "CIDEr")
top_spice = df_results.nlargest(top_n, "SPICE")

# 상위 조합 선택
top_candidates = pd.concat([top_bertscore, top_meteor, top_cider, top_spice]).drop(columns=["Generated Texts"])

# 가장 많이 등장한 조합 찾기
best_config = top_candidates.value_counts().idxmax()

print("\n=== 최적의 하이퍼파라미터 조합2 ===")
print(best_config)



=== 최적의 하이퍼파라미터 조합2 ===
(3, 1.2, 1.5, 2, 0.8403462916612625, 0.4305769386645354, 1.8124951037942274, 0.2843705353785999, 3.5470953140585593, 3.562945617417329, 3.6298761251801146, 3.324118363484937, 3.5222261468398997, 0.8419472173746563)


In [ ]:
# 방법3. 모든 지표의 평균을 계산해서 가장 높은 평균을 가진 조합

df_results["Mean_Score"] = df_results[["BERTScore", "METEOR", "CIDEr", "SPICE"]].mean(axis=1)
best_config = df_results.sort_values(by="Mean_Score", ascending=False).iloc[0]

print("\n=== 최적의 하이퍼파라미터 조합3 (평균 점수 기준) ===")
print(best_config)


=== 최적의 하이퍼파라미터 조합3 (평균 점수 기준) ===
num_beams                                                               3
length_penalty                                                        0.8
repetition_penalty                                                    1.5
no_repeat_ngram_size                                                    3
BERTScore                                                        0.835566
METEOR                                                            0.42106
CIDEr                                                            1.837091
SPICE                                                            0.307842
Generated Texts         [최승희는 낡은 사진첩 하나를 꺼냈어요. 할머니, 우리나라로 와서 옛날이야기를 들려...
BERTScore_z                                                      3.439738
METEOR_z                                                         3.465633
CIDEr_z                                                          3.683266
SPICE_z                                                          3.623154
Fi

In [ ]:
### 확인

import random
from torch.utils.data import Subset, DataLoader

# 샘플
sample_indices = random.sample(range(len(val_dataset)), 3)
sample_dataset = Subset(val_dataset, sample_indices)
sample_dataloader = DataLoader(sample_dataset, batch_size=8, shuffle=False, collate_fn=data_collator)

# 최적 하이퍼파라미터 조합(방법3 이용)
best_params = {
    "num_beams": 3,
    "length_penalty": 0.8,
    "repetition_penalty": 1.5,
    "no_repeat_ngram_size": 3
}

# 모델 예측 함수
def generate_predictions(model, tokenizer, dataloader, params):
    model.eval()
    predictions = []
    references = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128,
                num_beams=params["num_beams"],
                length_penalty=params["length_penalty"],
                repetition_penalty=params["repetition_penalty"],
                no_repeat_ngram_size=params["no_repeat_ngram_size"],
                early_stopping=True
            )

            # 예측 및 정답 디코딩
            generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            reference_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

            predictions.extend(generated_texts)
            references.extend(reference_texts)

    return predictions, references

# 예측 실행
preds, refs = generate_predictions(model, tokenizer, sample_dataloader, best_params)

# 예측 & 정답 출력
for i, (pred, ref) in enumerate(zip(preds, refs)):
    print(f"\n[샘플 {i+1}]")
    print(f"예측: {pred}")
    print(f"정답: {ref}")
    print("-" * 80)



[샘플 1]
예측: 슈베르트는 열세 살 때 아버지가 돌아가시는 바람에 오케스트라 단원이 되기로 했어요. 하지만 슈베르트의 머릿속은 온통 음악 생각뿐이었어요. 음악에 대한 열정은 누구보다 강했지요. 슈베르트, 너는 음악가가 될 자격이 충분해! 아버지는 슈베르트를 칭찬했어요.
정답: 슈베르트는 음악 공부를 열심히 했어요. 수학, 역사, 지리 공부도 게을리해서는 안 된다. 아버지 말씀이 마음에 걸렸지만, 음악 공부가 너무너무 좋았어요. 슈베르트는 학교 오케스트라 단원으로 활동하면서 좋은 친구를 많이 사귀었어요.
--------------------------------------------------------------------------------

[샘플 2]
예측: 진솔이는 상어 이빨을 싹싹 닦아 주었어요. 이빨을 잘 닦지 않아서 이빨이 더러워진 것 같았어요. 동물 친구들도 상어의 이빨을 고쳐 주는 방법을 알려 주었어요!
정답: 휴, 상어야. 이빨을 닦지 않아서 입 속에 충치 세균과 감기 세균이 가득해. 진솔이는 커다란 칫솔로 상어의 이빨을 닦아 주었어요. 진솔이는 동물 친구들에게 이빨 닦는 방법을 알려 주었어요.
--------------------------------------------------------------------------------

[샘플 3]
예측: 장글대는 밥을 먹으러 가면서 주인에게 일러두었어. 한양은 눈 뜨고도 코가 베이는 곳이니, 정신 바짝 차리고 있거라!
정답: 드디어 한양에 도착했어. 주인은 밥을 먹으러 가면서 장글대에게 단단히 일러두었어. 한양은 눈 뜨고도 코가 베이는 곳이니, 정신 바짝 차리고 있거라!
--------------------------------------------------------------------------------
